# Post code matching


This notebook contains code that retrieves addresses and ids that contain a postcode which matches with southwestern German postcodes by the first two letters. 
Non-German addresses are mostly excluded. Due to the unconsistent string patterns for addresses, some non-German addresses remain in the result and have to be excluded manually.  

The function can be applied to non-German post codes, too.

In [4]:
#import
import pandas as pd
import numpy as np
import csv
from neo4j import GraphDatabase
from myconfig import *

In [5]:
class Neo4jConnection:
    
    '''
    This class handles the connection to the Neo4j DBMS. 
    Source: https://towardsdatascience.com/neo4j-cypher-python-7a919a372be7
    
    Initialization variables 
    uri: URI used to connect to a database from the client
    user: DBMS username
    pwd: DBMS password
    
    Uri, user, and pwd are noted in a separate file called 'myconfig.py' and are imported.
    
    '''
    
    def __init__(self, uri, user, pwd):
        self.__uri=uri
        self.__user=user
        self.__pwd=pwd
        self.__driver = None
        
        # open driver
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
            
    def query(self, query, db=None):
        #check if driver is initialized
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        #try session creation and run query
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [6]:
def add_pc(pc_list, new_pc):
    '''
    Function to add new post code.
    '''
    if type(new_pc) == str:
        pc_list.append(new_pc)
    else:
        try:
            pc_list.append(str(new_pc))
        except:
            print("An exception occurred.")

In [7]:
def construct_query(num, DE=True):
    '''
    Function to construct a query to match for a German five digit post code or a post code of any length that start with num. 
    '''
    if DE:
        numm=str(5-len(num))
        query = "MATCH (a:Address) WHERE a.address =~ '(\W|\S)*\D+" + num + "\d{" + numm + "}(\S|\W)*' RETURN a.node_id as id, a.address AS address"
        return query
    else:
        query = "MATCH (a:Address) WHERE a.address =~ '(\W|\S)*\D+" + num + "(\S|\W)*' RETURN a.node_id as id, a.address AS address"
        return query

In [8]:
def clean_df(df):
    '''
    Function to clean df from Non-German addresses
    '''
    
    df['address'] = df['address'].str.lower()
    df=df[df['address'].str.match('(\W|\S)*texas(\W|\S)*')== False]
    df=df[df['address'].str.match('(\W|\S)*singapore(\W|\S)*')== False]
    df=df[df['address'].str.match('(\W|\S)*france(\W|\S)*')== False]
    df=df[df['address'].str.match('(\W|\S)*nederland(\W|\S)*')== False]
    df=df[df['address'].str.match('(\W|\S)*\W+tx\W*(\W|\S)*')== False]
    df=df[df['address'].str.match('(\W|\S)*malaysia(\W|\S)*')== False]
    df=df[df['address'].str.match('(\W|\S)*las vegas(\W|\S)*')== False]
    df=df[df['address'].str.match('(\W|\S)*mauritius(\W|\S)*')== False]
    df=df[df['address'].str.match('(\W|\S)*dubai(\W|\S)*')== False]
    df=df[df['address'].str.match('(\W|\S)*emirates(\W|\S)*')== False]
    df=df[df['address'].str.match('(\W|\S)*paris(\W|\S)*')== False]
    df=df[df['address'].str.match('(\W|\S)*india(\W|\S)*')== False]
    df=df[df['address'].str.match('(\W|\S)*ukraine(\W|\S)*')== False]
    df=df[df['address'].str.match('(\W|\S)*thailand(\W|\S)*')== False]
    df=df[df['address'].str.match('(\W|\S)*mexico(\W|\S)*')== False]
    df=df[df['address'].str.match('(\W|\S)*israel(\W|\S)*')== False]
    df=df[df['address'].str.match('(\W|\S)*nevada(\W|\S)*')== False]
    df=df[df['address'].str.match('(\W|\S)*italy(\W|\S)*')== False]
    return df

In [9]:
def get_addresses(pc_list):
    #generate empty pandas df
    addresses_df = pd.DataFrame(columns = ['id', 'address'])
    #create connection to graph dbms
    c=Neo4jConnection(uri, user, pwd)
    #for each two-digit combination
    for pc in pc_list:
        q=construct_query(pc) #construct query
        rows = pd.DataFrame([dict(_) for _ in c.query(q, db='neo4j')]) #get info from db
        addresses_df=pd.concat([addresses_df, rows]) #add info to df
    c.close()
    addresses_df=addresses_df.reset_index()
    return addresses_df

In [10]:
plz_list=["78", "88", "79", "72", "89", "77"]

In [13]:
df_sus = get_addresses(plz_list)
df_sus = clean_df(df_sus)


In [14]:
df_sus

,index,id,address
17,17,14012487,2 rue des peupliers 78450 chavenay.
22,22,14016179,413 united nation rd. p.o. box box 78282 dar e...
42,42,14063853,"p.o. box 785700 sandton 2146, south africa"
43,43,14064650,privada the bear no. 10; cond res la loma club...
44,44,14067562,rod. mário andreazza; 1800; bairro guarita; va...
...,...,...,...
1241,309,58117949,zwiebelbuhndstrasse 2 rheinmunster d-77836
1247,315,58133190,"clle villas tropical, km 14.5 lte, zona hotele..."
1249,317,58137178,obere haupstr 19 77971 kippenheim
1254,322,240111855,"petra lekovica 2/9l, 77030 beograd- cukarica, ..."


In [ ]:
df_sus.to_csv('data/df_sus.csv', encoding='utf-16', index=False)